In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install sktime neuralforecast pmdarima darts numpy==1.26.4

In [3]:
import os
os.chdir('engagement-prediction/src')

In [4]:
storage_path='/content/drive/MyDrive/thesis/'

In [5]:
import datetime as dt
import pandas as pd
import numpy as np
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator

In [6]:
input_file=os.path.join(storage_path,'data','influencer_sample010525_weekly.csv')

output_file_summary=os.path.join(storage_path,'output',f"summary_df_{dt.datetime.now():%y%m%d%H%M}.csv")

output_file_complete=os.path.join(storage_path,'output',f"summary_complete_df_{dt.datetime.now():%y%m%d%H%M}.csv")

In [7]:

h=4
window=12
test=True

sources=['instagram-content','twitter-tweets','tiktok-content','facebook-content','youtube-content',
 'tiktok-content-oauth','instagram-content-oauth','tiktok-content-public']
#models_eval=['AutoARIMA', 'Theta', 'ETS', 'Prophet', 'AutoREG(2)', 'AutoREG(3)', 'AutoREG(4)', 'AutoREG(5)', 'AR(2)', 'AR(3)', 'AR(4)', 'AR(5)', 'MA(2)', 'MA(3)', 'MA(4)', 'MA(5)', 'Neural Forecast LTSM', 'Neural Forecast RNN']
models_eval=['AutoARIMA', 'Theta', 'ETS', 'Prophet', 'AutoREG(2)', 'AutoREG(3)', 'AutoREG(4)', 'AutoREG(5)', 'AR(2)', 'AR(3)', 'AR(4)', 'AR(5)', 'MA(2)', 'MA(3)', 'MA(4)', 'MA(5)']
df= pd.read_csv(input_file).query('source in @sources').reset_index(drop=True)
df['published_date'] = pd.to_datetime(df['published_date_w'])
df


,published_date_w,channel_uid,n_posts,engagements_sum,video_views_sum,shares_sum,comments_sum,first_audience,last_audience,source,tier,mean_audience,engagements_rate_sd,posts_per_day,normalized_engagement_rate,min_published_date_w,audience_growth,first_non_zero_growth_date,published_date
0,2017-09-10,d0334d17ea133c5aac89e40ce50404d8,5,90,0,0,2,29295.0,29297.0,instagram-content,"10,000–49,999",2.929580e+04,0.000655,0.714286,0.003072,2016-01-24,0.000068,2017-09-10,2017-09-10
1,2017-09-17,d0334d17ea133c5aac89e40ce50404d8,9,246,1153,0,15,29297.0,29289.0,instagram-content,"10,000–49,999",2.929256e+04,0.001195,1.285714,0.008398,2016-01-24,-0.000273,2017-09-10,2017-09-17
2,2017-10-01,d0334d17ea133c5aac89e40ce50404d8,4,166,0,0,5,29280.0,29280.0,instagram-content,"10,000–49,999",2.928000e+04,0.001152,0.571429,0.005669,2016-01-24,-0.000307,2017-09-10,2017-10-01
3,2017-10-08,d0334d17ea133c5aac89e40ce50404d8,5,98,0,0,4,29280.0,29270.0,instagram-content,"10,000–49,999",2.927600e+04,0.000583,0.714286,0.003347,2016-01-24,-0.000342,2017-09-10,2017-10-08
4,2017-10-22,d0334d17ea133c5aac89e40ce50404d8,2,15,0,0,2,29298.0,29298.0,instagram-content,"10,000–49,999",2.929800e+04,0.000024,0.285714,0.000512,2016-01-24,0.000957,2017-09-10,2017-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19609,2025-03-16,1c8afac3e44638cab68212f6f634f6d8,1,24699,0,0,296,2473766.0,2473766.0,instagram-content,"500,000+",2.473766e+06,NaN,0.142857,0.009984,2019-08-04,0.000000,2021-10-31,2025-03-16
19610,2025-03-16,6833f3e4b5753906a72ff0fbbdb53151,2,223,0,6,19,317000.0,317000.0,tiktok-content,"100,000–499,999",3.170000e+05,0.000199,0.285714,0.000703,2019-11-24,0.000000,2020-10-25,2025-03-16
19611,2025-03-16,a8b4713fd3393bbcaa4df879a42d1efc,1,19,0,0,0,9782.0,9782.0,instagram-content,"1,000–9,999",9.782000e+03,NaN,0.142857,0.001942,2017-12-24,0.000000,2020-04-05,2025-03-16
19612,2025-03-16,e04c6702e00a321a8f98624ffe7fd23e,2,781,0,154,15,1900000.0,1900000.0,tiktok-content,"500,000+",1.900000e+06,0.000185,0.285714,0.000411,2021-10-31,0.000000,2022-03-20,2025-03-16


In [8]:
df['source'].value_counts()

,count
source,
instagram-content,14224
tiktok-content,3357
facebook-content,1082
youtube-content,951


In [9]:
ids_test=['b95a899244573cf38802aa520ef3a82b']
if test:
    df_temp=df.query('channel_uid in @ids_test')
else:
    df_temp=df.copy()
df_temp['source'].unique()

array(['instagram-content'], dtype=object)

In [10]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid')

In [ ]:
multi_eval.run(h=h, initial_window=window, step_length=1,models=models_eval,metric_cols=['mean_audience'])

summary_df = multi_eval.get_summary()
summary_complete_df = multi_eval.get_summary_complete()

Evaluating models:   0%|          | 0/16 [00:00<?, ?it/s]

2025-05-18 13:18:15,443 Evaluating channel: b95a899244573cf38802aa520ef3a82b


INFO:metrics_forecast:Evaluating channel: b95a899244573cf38802aa520ef3a82b


2025-05-18 13:18:15,449 Evaluating metric: mean_audience


INFO:metrics_forecast:Evaluating metric: mean_audience


2025-05-18 13:18:15,451 Evaluating AutoARIMA for b95a899244573cf38802aa520ef3a82b/mean_audience


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed

In [ ]:

summary_df.to_csv(output_file_summary, index=False)
summary_df

In [ ]:

summary_complete_df.to_csv(output_file_complete, index=False)
summary_complete_df